In [2]:
import anndata as ad
import scipy.io
import pandas as pd
import os
import anndata as ad

In [15]:
data_path = os.path.join("/lustre/groups/ml01/workspace/laura.martens/moretti_colab/archr_final", "Epicardiods", "Anndata")

In [7]:
data_path

'/storage/groups/ml01/workspace/laura.martens/moretti_colab/archr_new/Epicardiods/Anndata'

In [8]:
with open(os.path.join(data_path, 'matrix.mtx'), 'rb') as mm:
    x = scipy.io.mmread(mm).T.tocsr()

In [9]:
x

<30294x378157 sparse matrix of type '<class 'numpy.int64'>'
	with 278766147 stored elements in Compressed Sparse Row format>

In [11]:
obs = pd.read_csv(os.path.join(data_path, 'obs.csv'), sep=',')

In [13]:
var = pd.read_csv(os.path.join(data_path, 'var.csv'), sep=',')

In [15]:
var

,seqnames,start,end,width,strand,score,replicateScoreQuantile,groupScoreQuantile,Reproducibility,GroupReplicate,distToGeneStart,nearestGene,peakType,distToTSS,nearestTSS,GC,idx,N
1,chr1,758176,758676,501,*,3.06380,0.249,0.055,2,C9._.MUC26651_1234,4475,LINC00115,Distal,4475,uc001aau.3,0.5569,1,0
2,chr1,761967,762467,501,*,6.66919,0.686,0.369,2,C1._.MUC26654_1234,684,LINC00115,Promoter,684,uc001aau.3,0.5649,2,0
3,chr1,762680,763180,501,*,260.96500,0.967,0.946,5,C5._.MUC26651_1234,27,LINC00115,Promoter,27,uc001aau.3,0.6886,3,0
4,chr1,766597,767097,501,*,133.51800,0.901,0.849,2,C19._.MUC26652_1234,3875,LINC01128,Intronic,3668,uc021oeh.1,0.4092,4,0
5,chr1,767434,767934,501,*,20.35000,0.817,0.669,2,C18._.MUC26652_1234,4712,LINC01128,Intronic,4505,uc021oeh.1,0.5509,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378153,chrX,154995895,154996395,501,*,12.20470,0.474,0.232,2,C19._.MUC26652_1234,153522,TMLHE,Distal,1305,uc004cin.3,0.3812,9543,0
378154,chrX,154997000,154997500,501,*,91.36150,0.938,0.890,2,C18._.MUC26651_1234,154627,TMLHE,Distal,200,uc004cin.3,0.5409,9544,0
378155,chrX,154997632,154998132,501,*,7.70711,0.450,0.155,2,C18._.MUC26651_1234,155259,TMLHE,Distal,430,uc004cin.3,0.4192,9545,0
378156,chrX,154999234,154999734,501,*,15.55320,0.538,0.308,2,C19._.MUC26652_1234,156861,TMLHE,Distal,2032,uc004cin.3,0.3772,9546,0


In [19]:
var.index = var.seqnames + ":" + var.start.astype(str) + "-" + var.end.astype(str)

In [22]:
adata = ad.AnnData(X=x, obs=obs, var=var)
adata.var_names_make_unique()

/tmp/ipykernel_6232/1750927739.py:1: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=x, obs=obs, var=var)


In [26]:
adata.write(os.path.join(data_path, 'adata.h5ad'))

In [27]:
import scanpy as sc

In [28]:
sc.pp.filter_genes(adata, min_cells = int(adata.shape[0]*0.01))

In [30]:
adata.write(os.path.join(data_path, 'adata_filtered.h5ad'))

# Load RNA seq data

In [17]:
data_path = os.path.join("/lustre/groups/ml01/workspace/laura.martens/moretti_colab/transfer_data")

In [4]:
counts = pd.read_csv(os.path.join(data_path, "counts.csv"))

In [5]:
obs = pd.DataFrame(counts['clusters'], index = counts.index)

In [6]:
counts = counts.drop('clusters', axis =1)

In [7]:
var = pd.DataFrame(counts.columns.values, index=counts.columns.values, columns=["gene"])

In [8]:
adata = ad.AnnData(counts, obs=obs, var=var)

In [9]:
adata

AnnData object with n_obs × n_vars = 35499 × 24353
    obs: 'clusters'
    var: 'gene'

In [10]:
adata.obs['day'] = adata.obs_names.str.split('_').str[-1]

In [11]:
day_map = {
    '1':'2',
    '2': '3',
    '3': '4',
    '4': '5',
    '5': '7',
    '6': '10',
    '7': '15'
}

In [12]:
adata.obs['day'] = adata.obs.day.map(day_map)

In [18]:
adata.write(os.path.join(data_path, "Day2_15_RNAseq.h5ad"))

In [14]:
adata.obs

,clusters,day
AAACCCAAGAGCAAGA-1_5,22,7
AAACCCAAGCCGCACT-1_7,1,15
AAACCCAAGCCGTTGC-1_5,0,7
AAACCCAAGCGAGAAA-1_6,21,10
AAACCCAAGCTCCATA-1_1,2,2
...,...,...
TTTGTTGTCGGATACT-1_5,27,7
TTTGTTGTCGTCAAAC-1_4,8,5
TTTGTTGTCGTTCCCA-1_5,0,7
TTTGTTGTCTCTCTAA-1_6,6,10
